In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os

In [60]:
url = 'https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220818.html'

In [61]:
ssm = requests.get(url)

In [62]:
ssm.status_code

200

In [63]:
s = BeautifulSoup(ssm.text, 'lxml').find_all('tr')

In [64]:
len(s)

11

In [81]:
lugar = []
fecha = []
lat = []
lng = []
profundidad = []
magnitud = []

In [82]:
for i in range(2,len(s)):
    
    lugar.append(s[i].find_all('td')[0].text[19:])
    fecha.append(s[i].find_all('td')[1].text) # Fecha
    lat.append(float(s[i].find_all('td')[2].text.split()[0]))
    lng.append(float(s[i].find_all('td')[2].text.split()[1]))# Latitud y longitud
    profundidad.append(s[i].find_all('td')[3].text) # Profundidad
    magnitud.append(s[i].find_all('td')[4].text) #Magnitud 
    

In [86]:
sismos_dia = pd.DataFrame(zip(lugar,fecha,lat,lng,profundidad,magnitud),
                         columns = ['lugar','fecha','latitud','longitud','profundidad','magnitud'])

In [87]:

sismos_dia

,lugar,fecha,latitud,longitud,profundidad,magnitud
0,22 km al SO de Mina Collahuasi,2022-08-18 22:22:04,-20.911,-68.850,101 km,2.5 Ml
1,26 km al SE de Mina Collahuasi,2022-08-18 20:17:20,-20.989,-68.480,133 km,3.0 Ml
2,50 km al S de Pica,2022-08-18 17:19:00,-20.938,-69.262,75 km,4.2 Ml
3,158 km al O de Tocopilla,2022-08-18 12:16:43,-21.610,-71.646,22 km,2.8 Ml
4,25 km al E de Pica,2022-08-18 04:54:24,-20.542,-69.097,105 km,2.7 Ml
5,63 km al S de Putre,2022-08-18 03:28:12,-18.765,-69.602,103 km,2.7 Ml
6,27 km al SE de Rancagua,2022-08-18 02:10:22,-34.372,-70.573,114 km,5.3 Mww
7,65 km al E de Socaire,2022-08-18 00:49:53,-23.430,-67.281,255 km,3.0 Ml
8,86 km al S de Socaire,2022-08-18 00:12:56,-24.321,-68.160,258 km,2.9 Ml


In [12]:
sismos_dia.to_csv('sismos_dia.csv')

In [ ]:
s[2].td.text # Fecha Direccion

s[2].find_all('td')[0].text # Fecha direccion
s[2].find_all('td')[1].text # Fecha
s[2].find_all('td')[2].text # Latitud y longitud
s[2].find_all('td')[3].text # Profundidad
s[2].find_all('td')[4].text #Magnitud 


IndexError: list index out of range

### Generalizacion de URL en multiples fecha

In [40]:
fechas = pd.date_range(start='1/1/2022', end='01/14/2022')

In [41]:
#Estructura URL
# https://www.sismologia.cl/sismicidad/catalogo/AAAA/MM/AAAAMMDD.html
for i in fechas:
    print(i.strftime('%Y/%m/%Y%m%d'))


2022/01/20220101
2022/01/20220102
2022/01/20220103
2022/01/20220104
2022/01/20220105
2022/01/20220106
2022/01/20220107
2022/01/20220108
2022/01/20220109
2022/01/20220110
2022/01/20220111
2022/01/20220112
2022/01/20220113
2022/01/20220114


## Generacion funcion scraper por dia



In [113]:
def get_sismos(fecha):
    fecha_data = pd.to_datetime(fecha)
    url = 'https://www.sismologia.cl/sismicidad/catalogo/'+fecha_data.strftime('%Y/%m/%Y%m%d')+'.html'
    print(url)
    ssm = requests.get(url)
    s = BeautifulSoup(ssm.text, 'lxml').find_all('tr')
    
    lugar = []
    fecha = []
    lat = []
    lng = []
    profundidad = []
    magnitud = []
    
    for i in range(2,len(s)):
    
        lugar.append(s[i].find_all('td')[0].text[19:])
        fecha.append(s[i].find_all('td')[1].text) # Fecha
        lat.append(float(s[i].find_all('td')[2].text.split()[0]))
        lng.append(float(s[i].find_all('td')[2].text.split()[1]))
        profundidad.append(s[i].find_all('td')[3].text) 
        magnitud.append(s[i].find_all('td')[4].text)  
    
    sismos_dia = pd.DataFrame(zip(lugar,fecha,lat,lng,profundidad,magnitud),
                              columns = ['lugar','fecha','latitud','longitud','profundidad','magnitud'])
    
    nombre_archivo = 'sismos_'+fecha_data.strftime('%Y%m%d')+'.csv'

    
    # Esta seccion guarda el archivo CSV en la carpeta "data"
    file_dir = os.path.dirname(os.path.abspath("__file__"))
    csv_folder = 'data'
    file_path = os.path.join(file_dir, csv_folder, nombre_archivo)
    sismos_dia.to_csv(file_path, index=False)
    print('Archivo guardado', nombre_archivo)
    
    

In [112]:
get_sismos('08/20/2022')

https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220820.html
Archivo guardado sismos_20220820.csv


In [ ]:
target = pd.date_range(start='05/01/2022', end='05/07/2022')

for i in target:
    get_sismos(i)
    sleep(1)

https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220501.html
Archivo guardado sismos_20220501.csv
https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220502.html
Archivo guardado sismos_20220502.csv
https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220503.html
Archivo guardado sismos_20220503.csv
https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220504.html
Archivo guardado sismos_20220504.csv
https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220505.html
Archivo guardado sismos_20220505.csv
https://www.sismologia.cl/sismicidad/catalogo/2022/05/20220506.html
Archivo guardado sismos_20220506.csv


## Utilizando la funcion en un rango de fechas